In [1]:
import sys
import importlib
sys.path.insert(0,'../')
import numpy as np
import netket as nk
from scripts import functions as f
importlib.reload(f)

from scripts import new_dynamics, new_dynamics2
import os
currentpath = os.getcwd()
parentdir = os.path.dirname(currentpath)

In [2]:
from scripts import Dimer_RBM

In [3]:
length=[4, 4]
V = 1.0
h = 1.0
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)


ham = f.dimer_hamiltonian(V = V, h = h ,length=np.array(length))
op_transition1 = f.dimer_flip1(length = np.array(length))


hex_ = nk.machine.new_hex(np.array(length))

ma = nk.machine.RbmDimer(hi, hex_, alpha = 2, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float, reverse=True, half=True)
ma.init_random_parameters(seed=1234)

yoyo
2


In [4]:
ma._ws[:] = 0

In [5]:
ma._set_bare_parameters(
    ma._a, ma._b, ma._w, ma._as, ma._bs, ma._ws, ma._autom, ma._z2
)

In [6]:
# N = 1000000
# temp = np.random.choice([-1,1],hex_.size*N).reshape(N, hex_.size)
# ind = hex_.is_dimer_basis(temp)

In [7]:

# create save folder.


n_jobs = 4

g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)

op = f.dimer_hamiltonian(h, V,np.array(length))
op_transition = f.dimer_flip1(length = np.array(length))
hex_ = nk.machine.new_hex(np.array(length))




sweep_size = 1000
sa_mul = nk.sampler.DimerMetropolisLocal_multi(machine=ma, op=op_transition, length = length, n_chains=1, sweep_size = sweep_size, kernel = 1, n_jobs=n_jobs)
sa_mul.reset()
sa_mul.generate_samples(1000) # discard the begginings of metropolis sampling.



yoyo
number of core : 4


array([[[-1., -1., -1., ..., -1., -1., -1.]],

       [[-1.,  1.,  1., ...,  1.,  1.,  1.]],

       [[-1., -1.,  1., ..., -1., -1., -1.]],

       ...,

       [[ 1.,  1., -1., ...,  1.,  1., -1.]],

       [[-1., -1., -1., ..., -1., -1., -1.]],

       [[-1., -1.,  1., ..., -1., -1.,  1.]]])

In [8]:
n_samples_ = 3000
samples_state = sa_mul.generate_samples(int(n_samples_ / 1))
samples_state = samples_state.reshape(-1, ma.hilbert.size)
print('get samples')

get samples


In [9]:
import scripts.dynamics as Dynamics
import scripts.dynamics_4 as sdy

import scripts
importlib.reload(sdy)

<module 'scripts.dynamics_4' from '../scripts/dynamics_4.py'>

In [10]:
t_list = np.arange(0,5,0.1)

# d = Dynamics.new_dynamics(op, ma)
# %time P_ = d.multiprocess(samples_state, 100, 4) 
# P = P_[0]
# T = P_[1]
# P_list = f.process_P(P, T, t_list)

d = sdy.new_dynamics_one(op, ma)
P_list = d.multiprocess(samples_state, t_list,4) 

In [13]:
edge_corr1 = np.array([ 0.5     ,  0.      ])
edge_corr2 = np.array([ 0.25    , -0.433013])
edge_corr3 = np.array([ 0.75    , -0.433013])

In [14]:
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)
hex_ = nk.machine.new_hex(np.array(length))


x = np.arange(length[0])
y = np.arange(length[1])

xx, yy = np.meshgrid(x,y)
xx = xx.reshape(-1)
yy = yy.reshape(-1)

operators_list = []

for base_edge in [edge_corr1, edge_corr2, edge_corr3]:
    edge_coor_array = hex_.ProcessPeriodic(xx[:,None] * hex_.a1[None,:]+ yy[:,None]  * hex_.a2[None,:] + base_edge)
    edges = hex_.edge_coor_to_lattice(edge_coor_array)
    colors = hex_.get_edge_color(edges)

    operators_list.append(f.return_spin_corr(hi, edges, colors))

In [15]:
edge_coors = np.array([[ 0.5     ,  0.      ],
                       [ 0.25    , -0.433013],
                       [ 0.75    , -0.433013]])

a = np.tan((1/6)*np.pi) * 1/2
edge_coors_prime = np.array([
    hex_.a((1/2)*np.pi) * a,
    hex_.a(np.pi*(2/3)+(1/2)*np.pi) * a,
    hex_.a(np.pi*(4/3)+(1/2)*np.pi) * a,
])
edge_coors_prime = np.round(edge_coors_prime,10)

In [16]:
K = np.zeros((2*length[0],2*length[1],2))

In [17]:
A = np.array([
    hex_.R1,
    hex_.R2
])

In [18]:
[b1, b2] = np.linalg.inv(A.T)
b = np.array([b1, b2])
G = edge_coors @ b.T

In [19]:
K = np.zeros((2*length[0],2*length[1],2))
for l1 in range(K.shape[0]):
    for l2 in range(K.shape[1]):
        K[l1,l2] = hex_.b1 * 2*np.pi*l1/K.shape[0] + hex_.b2 * 2*np.pi*l2/K.shape[1]

In [20]:
uni = np.exp(1j*np.einsum('ijk,lk->lij',K,edge_coors_prime))

In [22]:
dimer_corr[a,b]

array([[[1.        , 0.92866667, 0.876     , 0.824     , 0.79133333,
         0.76133333, 0.734     , 0.718     , 0.696     , 0.68866667,
         0.67266667, 0.65666667, 0.63866667, 0.63466667, 0.62133333,
         0.60733333, 0.59866667, 0.58466667, 0.57      , 0.55333333,
         0.548     , 0.544     , 0.55066667, 0.55533333, 0.54533333,
         0.54066667, 0.536     , 0.53533333, 0.53733333, 0.528     ,
         0.53933333, 0.54      , 0.54066667, 0.53466667, 0.534     ,
         0.528     , 0.518     , 0.51933333, 0.52533333, 0.53866667,
         0.54      , 0.534     , 0.53533333, 0.53466667, 0.53733333,
         0.52733333, 0.50466667, 0.51066667, 0.51133333, 0.50666667],
        [0.554     , 0.53466667, 0.52466667, 0.51133333, 0.50933333,
         0.502     , 0.488     , 0.47333333, 0.47666667, 0.48133333,
         0.48      , 0.48266667, 0.486     , 0.49133333, 0.48733333,
         0.476     , 0.48066667, 0.47866667, 0.47066667, 0.46333333,
         0.45533333, 0.45933333, 

In [23]:
ma.log_val(samples_state[0].reshape(1,-1))

array([0.+0.j])

In [24]:
samples_state[0]

array([-1., -1., -1.,  1., -1., -1.,  1.,  1.,  1., -1., -1.,  1.,  1.,
       -1.,  1.,  1., -1., -1., -1.,  1., -1., -1., -1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.])

In [28]:
T, color = hex_.autom(reverse=True, half=True)
from numba import njit

In [29]:
@njit
def _permute(x, T, color):
    x_ = np.zeros((T.shape[0], x.shape[-1]))
    
    for j in range(len(x_)):
        for i in range(len(x)):
            x_[j,i] = x[T[j,i]] * color[i]
    return x_

In [30]:
@njit
def permute(X, T, color):
    
    X_ = np.zeros((X.shape[0]*2, T.shape[1], X.shape[-1]))
    
    for a in range(2):
        for i in range(len(X)):
            X_[2*i + a] = _permute(X[i], T[a], color[a])
    return X_.reshape(-1,X.shape[-1])

In [39]:
new_P_list = np.zeros((P_list.shape[0], P_list.shape[1] * T.shape[0] * T.shape[1], P_list.shape[-1]))

for t in range(len(new_P_list)):
    new_P_list[t] = permute(P_list[t], T, color)

In [42]:
dimer_corr = np.zeros((3,3,length[0],length[0],len(t_list)), dtype = np.float64)

for a in range(3):
    for b in range(3):
        dimer_corr[a,b], _, _ = f.cal_dimer_corr(operators_list[a][0], operators_list[b], new_P_list, hex_, t_list)

In [43]:
from numba import njit
@njit
def dimer_fourier2D(corr):
    
    assert corr.shape[0] == corr.shape[1]
    L = x.shape[0]
    
    momentum = np.zeros((L*2, L*2, corr.shape[-1]), dtype=np.complex128)
    
    for m1 in range(2 * L):
        for m2 in range(2 * L):
            tmp = np.zeros(corr.shape[-1], dtype=np.complex128)
            for n1 in range(L):
                for n2 in range(L):
                    tmp += np.exp(-1j*(n1 * m1/L * 2 * np.pi + n2 * m2/L * 2 * np.pi)) * corr[n1,n2]
            momentum[m1,m2] = tmp/(2*L)
    
    return momentum


def dimer_fourier2D_sub(dimer_m_):
    assert dimer_m_.shape[0] == dimer_m_.shape[1] == 3
    assert dimer_m_.shape[2] == dimer_m_.shape[3] 
    
    edge_coors = np.array([[ 0.5     ,  0.      ],
                           [ 0.25    , -0.433013],
                           [ 0.75    , -0.433013]])
    [b1, b2] = np.linalg.inv(A.T)
    b = np.array([b1, b2])
    G = edge_coors @ b.T
    
    
    L = dimer_m_.shape[2]
    
    dimer_momentum = np.zeros((L,L,len(t_list)), dtype=np.complex128)
    
    for m1 in range(L):
        for m2 in range(L):
            tmp = np.zeros(dimer_m_.shape[-1],dtype=np.complex128)
            for a in range(3):
                for b in range(3):
                    tmp += np.exp(-1j * (m1* (2*np.pi) /L *(G[a,0] - G[b,0]) + m2* (2*np.pi) /L *(G[a,1] - G[b,1]))) * dimer_m_[a,b,m1,m2]
            dimer_momentum[m1,m2] = tmp / 3
    return dimer_momentum

In [44]:
dimer_momentum_ = np.zeros((3,3,8,8,len(t_list)),dtype=np.complex128)
for a in range(3):
    for b in range(3):
        dimer_momentum_[a,b] = dimer_fourier2D(dimer_corr[a,b])

In [45]:
dimer_momentum_[0,0][0,1] * 8

array([ 4.47000000e-01-1.66533454e-16j,  3.67333333e-01-2.77555756e-16j,
        3.05500000e-01-2.77555756e-16j,  2.61500000e-01-2.77555756e-16j,
        2.14500000e-01-2.22044605e-16j,  1.83666667e-01-2.22044605e-16j,
        1.58833333e-01-2.22044605e-16j,  1.37666667e-01-2.22044605e-16j,
        1.19833333e-01-2.22044605e-16j,  1.03833333e-01-2.22044605e-16j,
        8.80000000e-02-2.22044605e-16j,  7.51666667e-02-2.22044605e-16j,
        6.83333333e-02-2.22044605e-16j,  6.66666667e-02-2.22044605e-16j,
        5.71666667e-02-2.22044605e-16j,  4.91666667e-02-2.22044605e-16j,
        4.95000000e-02-2.22044605e-16j,  4.35000000e-02-2.22044605e-16j,
        3.41666667e-02-2.22044605e-16j,  3.33333333e-02-2.22044605e-16j,
        3.15000000e-02-2.22044605e-16j,  2.53333333e-02-2.22044605e-16j,
        2.16666667e-02-2.22044605e-16j,  1.58333333e-02-2.22044605e-16j,
        1.80000000e-02-2.22044605e-16j,  1.78333333e-02-2.22044605e-16j,
        1.56666667e-02-2.22044605e-16j,  1.41666667

In [47]:
from scipy import fft
fft.fft2(dimer_corr[0,0,:,:,0])

array([[8.154     -0.j, 0.447     +0.j, 0.        -0.j, 0.447     -0.j],
       [0.54933333+0.j, 0.557     +0.j, 0.26166667+0.j, 0.264     -0.j],
       [0.89266667-0.j, 0.92866667+0.j, 0.938     -0.j, 0.92866667-0.j],
       [0.54933333-0.j, 0.264     +0.j, 0.26166667-0.j, 0.557     -0.j]])

In [48]:
dimer_momentum = dimer_fourier2D_sub(dimer_momentum_)

dimer_momentum[0,1]

array([ 2.24555102e-02-6.24500451e-17j,  1.84533723e-02-7.40148683e-17j,
        1.53471105e-02-8.44232092e-17j,  1.31367247e-02-7.97972799e-17j,
        1.07756307e-02-8.03755211e-17j,  9.22668616e-03-7.40148683e-17j,
        7.97915781e-03-6.82324567e-17j,  6.91582828e-03-8.55796915e-17j,
        6.01995222e-03-7.51713506e-17j,  5.21617557e-03-7.05454214e-17j,
        4.42077159e-03-8.99165002e-17j,  3.77607573e-03-8.52905709e-17j,
        3.43279612e-03-9.28077060e-17j,  3.34906939e-03-8.26884857e-17j,
        2.87182700e-03-8.78926561e-17j,  2.46993867e-03-8.57242518e-17j,
        2.48668402e-03-8.76035355e-17j,  2.18526777e-03-8.47123297e-17j,
        1.71639806e-03-8.05200813e-17j,  1.67453469e-03-8.32667268e-17j,
        1.58243528e-03-7.77734358e-17j,  1.27264637e-03-8.25439254e-17j,
        1.08844755e-03-7.83516770e-17j,  7.95403979e-04-8.29053261e-17j,
        9.04248734e-04-8.76035355e-17j,  8.95876061e-04-7.82793969e-17j,
        7.87031306e-04-8.54351312e-17j,  7.11677245

In [88]:
tau_list = []
N = 2000
corr1 = []
corr2 = np.zeros((48,N),dtype=np.float64)
n = -1
import re
with open('RN4_4V1.0st.dat') as fe:
    for line in fe:
        
        if re.findall(r'#\d', line):
            i = 0
            n += 1
        else:
            try:
                match = re.findall(r'[\d,\.]+', line)
                tau_list.append(float(match[1]))
                corr2[n,i] = float(match[3])
                i+=1
            except:
                pass

In [65]:
par_path = '/Users/keisukemurota/Documents/Resaerch/Dimer_RBM/save/dat/'
path = par_path + "R[4,4]V1.0.dat"
from scripts import utils
importlib.reload(utils)
utils.write2dat(dimer_momentum, t_list, path)

In [78]:
(corr2[3][:200][None,None,:] - dimer_corr).mean(axis=-1).reshape(-1,order='F')

array([-0.05634956,  0.42758727,  0.00322997,  0.42758727, -0.0441282 ,
        0.42758727,  0.0046754 ,  0.42758727, -0.00115713, -0.00917361,
       -0.01261676, -0.00640277, -0.00570326, -0.0147057 , -0.01300102,
       -0.00211408, -0.02437084,  0.42758727, -0.01894938,  0.42758727,
       -0.02238292,  0.42758727, -0.02108872,  0.42758727, -0.00573926,
       -0.00756955, -0.01115713, -0.01382396, -0.00062284, -0.0066182 ,
       -0.0128662 , -0.01682243, -0.0241016 ,  0.42758727, -0.01379675,
        0.42758727, -0.02562104,  0.42758727, -0.01401078,  0.42758727,
       -0.0072511 , -0.01454327, -0.01759956, -0.00695526, -0.00468229,
       -0.01462088, -0.01405399, -0.00369133, -0.02069845,  0.42758727,
       -0.02181723,  0.42758727, -0.02030079,  0.42758727, -0.02049422,
        0.42758727,  0.00094381, -0.0047047 , -0.01026439, -0.01439945,
       -0.0041022 , -0.00484732, -0.01371034, -0.01257396])